In [1]:
import efaqa_corpus_zh
import pandas as pd
#from googletrans import Translator

In [2]:
# load database psychologique
database = list(efaqa_corpus_zh.load())
#print(database[10]['title'])

In [3]:
query_list = []
reponse_list = []
for i in range(len(database)-1):
    query_list.append(database[i]['title'])
    reponse_list.append(database[i]['chats'][0]['value'])
    #query_list.append(database[i]['title'])
    #reponse_list.append(database[i]['chats'][1]['value'])


In [4]:
dic_reponse_query = dict(zip(reponse_list,query_list))

In [15]:
query_frame = pd.DataFrame(query_list,columns=['Question'])
#query_frame.to_csv('question_ch.csv',header=True, index=False)
reponse_frame = pd.DataFrame(reponse_list,columns=['Reponse'])
#reponse_frame.to_csv('reponse_ch.csv',header=True, index=False)
query_reponse_frame = pd.concat([query_frame,reponse_frame],axis=1,ignore_index=False)
query_reponse_frame.to_csv('data/Q_R_ch.csv',sep=';',header=False, index=False)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import DistanceMetric

In [12]:
# obtenir une matrix de distance sur la question et la réponse
def get_distance(query,liste):
    matrix = [ [1]*len(liste) for i in range(len(liste))]
    #print(matrix)
    dist = DistanceMetric.get_metric('jaccard')
    for i in range(len(liste)):
        V = CountVectorizer()
        X = V.fit_transform([query, liste[i]]).toarray()
        distance = dist.pairwise(X)
        distance = distance.tolist()
        #distance = distance[1][0]
        matrix[i] = distance
    return matrix

def get_data(path):
    psycho_list = pd.read_csv(path,sep=';',names=['Question','Reponse'])
    psycho_frame = pd.DataFrame(psycho_list,columns=['Question','Reponse'])
    return psycho_frame


In [20]:
# BOT : calculer la similarité entre la requête et la question
def BOT_v1_multilingue():
    print("BOT : Hello, I am a baby of mom Shanshan and dad Cancan, I can speak French, Chinese and English, what language do you speak? please input fr or zh or en")
    langue = input()
    if langue == 'fr':
        path = 'data/Q_R_fr.csv'
        print("BOT : D'accord, qu'est-ce que je peux vous aider ?")
    if langue == 'zh':
        path = 'data/Q_R_ch.csv'
        print("BOT: 好的，请问有什么我可以帮您的吗")
    if langue == 'en':
        path = 'data/Q_R_en.csv'
        print("BOT: Okay, what can I help you with?")
    while True:
        query = input()
        if "bye" or '再见' or 'au revoir' in query:
            print('You : ',query)
            print("BOT : Bye!")
            break
        else :
            #query = '怎么重拾生活的兴趣？'
            print('You: ',query)
            psycho_frame = get_data(path)
            question_list = psycho_frame['Question'].to_list()
            reponse_list = psycho_frame['Reponse'].to_list()
            distance_query_question = get_distance(query,question_list)
            question_simi_index = distance_query_question.index(min(distance_query_question))
            reponse = reponse_list[question_simi_index]
            print("BOT : ",reponse)

In [21]:
BOT_v1_multilingue()

BOT : Hello, I am a baby of mom Shanshan and dad Cancan, I can speak French, Chinese and English, what language do you speak? please input fr or zh or en
BOT : D'accord, qu'est-ce que je peux vous aider ?
You :  au revoir
BOT : Bye!


In [22]:
BOT_v1_multilingue()

BOT : Hello, I am a baby of mom Shanshan and dad Cancan, I can speak French, Chinese and English, what language do you speak? please input fr or zh or en
BOT: 好的，请问有什么我可以帮您的吗
You :  再见
BOT : Bye!


In [23]:
BOT_v1_multilingue()

BOT : Hello, I am a baby of mom Shanshan and dad Cancan, I can speak French, Chinese and English, what language do you speak? please input fr or zh or en
BOT: Okay, what can I help you with?
You :  bye
BOT : Bye!


In [33]:
# BOT : calculer la similarité entre query/question et query/reponse
def BOT_v2_multilingue():
    print("BOT : Hello, I am a baby of mom Shanshan and dad Cancan, I can speak French, Chinese and English, what language do you speak? Please input fr or zh or en ^_^ ")
    langue = input()
    if langue == 'fr':
        path = 'data/Q_R_fr.csv'
        print("BOT : D'accord, qu'est-ce que je peux vous aider ?")
    if langue == 'zh':
        path = 'data/Q_R_ch.csv'
        print("BOT: 好的，请问有什么可以帮助您的呢？")
    if langue == 'en':
        path = 'data/Q_R_en.csv'
        print("BOT: Okay, what can I help you with?")
    while True:
        #query = 'psychologie'
        query = input()
        if "bye" in query:
            print('YOU: ',query)
            print("BOT : Bye !")
            break
        else :
            print('You: ',query)
            psycho_frame = get_data(path)
            question_list = psycho_frame['Question'].to_list()
            reponse_list = psycho_frame['Reponse'].to_list()
            distance_query_question = get_distance(query,question_list)
            distance_query_reponse = get_distance(query,reponse_list)
            #moyenne = mean(distance_query_question,distance_query_reponse)
            distance_question_min = min(distance_query_question)
            distance_reponse_min = min(distance_query_reponse)
            #distance = min(distance_question_min,distance_reponse_min)
            if distance_question_min < distance_reponse_min :
                question_simi_index = distance_query_question.index(distance_question_min)
                reponse = reponse_list[question_simi_index]
                print("BOT : ",reponse)
            else:
                question_simi_index = distance_query_reponse.index(distance_reponse_min)
                reponse = reponse_list[question_simi_index]
                print("BOT : ",reponse)

In [34]:
# test v1 français
BOT_v2_multilingue()

BOT : Hello, I am a baby of mom Shanshan and dad Cancan, I can speak French, Chinese and English, what language do you speak? Please input fr or zh or en ^_^ 
BOT: 好的，请问有什么可以帮助您的呢？
You:  您好我经常头疼想哭
BOT :  这样的议论是针对谁呢？
You:  关于离婚和财产切割问题
BOT :  这样的议论是针对谁呢？
YOU:  bye
BOT : Bye !
